Universidade Federal do Maranhão

Discente: Ruan Pactrick de Sousa e Sousa

Disciplina: Processamento de Imagens

Docente: Haroldo Gomes Barroso Filho

**Sistema de Classificação de Pets com IA**


Este projeto treina modelos de deep learning para identificar gatos e cães, depois classifica suas raças específicas usando:

*   TensorFlow
*   TensorFlow Lite



**1. Treinamento do Modelo Gato VS Cachorro**

**1.1 Carregamento do Dataset**

Utilizei o tfds.load() que é a forma recomendada pelo TensorFlow para baixar datasets. Então, isso permite com que eu faça o download automático e também já prepara os dados.

Esses dados estão sendo divididos em três partes:


*   80% para treino;
*   10% para validação;
*   10% para teste.

O que precisamos é que o programa identifique os dados como pares, ou seja, imagem e rótulo. Com isso, utilizei o as_supervised = True.

In [2]:
# import tensorflow as tf
# import tensorflow_datasets as tfds
# import os

# # Carregar o dataset "Cats vs Dogs" usando tensorflow_datasets
# dataset_name = "cats_vs_dogs"

# (train_ds, validation_ds, test_ds), metadata = tfds.load(
#     name=dataset_name,
#     split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
#     with_info=True,
#     as_supervised=True
# )

# print(f"Dataset '{dataset_name}' carregado com sucesso!")
# print(f"Número de classes: {metadata.features['label'].num_classes}")
# print(f"Nomes das classes: {metadata.features['label'].names}")
# print(f"Número de exemplos: {metadata.splits['train'].num_examples}")

**1.2 Preparação dos Dados**

Tive que resolver a questão das imagens serem de tamanhos diferentes, então foi preciso padronizar todas em 160x160 pixels. Além disso, é melhor por que o MobileNetV2 aceita receber imagens desse tamanho, então padronizei.

Depois disso, para que o modelo conseguisse agir mais rápido. Logo, utilizei uma normalização da "(imagem / 127.5) - 1" por que quanto mais próximo de zero, o modelo entende melhor.

In [3]:
# import tensorflow as tf
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras.models import Model

# IMG_SIZE = 160  # Tamanho que a MobileNet espera
# BATCH_SIZE = 32

# def format_image(image, label):
#     """Redimensiona e normaliza as imagens"""
#     image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
#     image = (image / 127.5) - 1  # Normaliza para [-1, 1]
#     return image, label

# # Aplicar formatação e organizar em lotes
# train_batches = train_ds.map(format_image).shuffle(1000).batch(BATCH_SIZE).prefetch(1)
# validation_batches = validation_ds.map(format_image).batch(BATCH_SIZE).prefetch(1)

**1.3 Criação da Arquitetura**

Utilizei a MobileNetV2 que já foi treinada com milhões de imagens e consegue identificar caracteristicas como: olho, orelhas e etc.

In [4]:
# # Usar MobileNetV2 pré-treinada no ImageNet
# base_model = MobileNetV2(
#     input_shape=(IMG_SIZE, IMG_SIZE, 3),
#     include_top=False,
#     weights='imagenet'
# )
# base_model.trainable = False  # Congelar os pesos existentes

# # Adicionar camadas de classificação personalizadas
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(128, activation='relu')(x)
# predictions = Dense(2, activation='softmax')(x)  # 2 classes: Gato e Cachorro

# model = Model(inputs=base_model.input, outputs=predictions)

**1.4 Compilação e Treinamento**

In [5]:
# # Compilar o modelo
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy']
# )

# # Treinar
# print("Iniciando treinamento...")
# history = model.fit(
#     train_batches,
#     epochs=5,
#     validation_data=validation_batches
# )

**1.5 Conversão para TensorFlow Lite**

Fiz essa conversão por que permite reduzir o tamanho dos arquivos e proporciona uma otimizaçao que permite rodar no celular ou no navegador.

In [6]:
# # Converter para TFLite
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

# # Salvar o arquivo
# with open('modelo_animais.tflite', 'wb') as f:
#     f.write(tflite_model)

# print("Arquivo 'modelo_animais.tflite' criado com sucesso!")

**2. Teste do Modelo Gato VS Cachorro**

Criei essa primeira parte apenas para fazer uma verificação inicial das imagens identificando se é cachorro ou gato.

In [7]:
# import tensorflow as tf
# import numpy as np
# import cv2
# from google.colab import files
# from IPython.display import Image, display

# # Configurações
# NOME_CLASSES = ['Gato', 'Cachorro']

# INFO_ANIMAIS = {
#     'Gato': "Temperamento: Independente e curioso.\nCuidados: Precisa de caixa de areia limpa e arranhadores.",
#     'Cachorro': "Temperamento: Leal e ativo.\nCuidados: Passeios diários e socialização são essenciais."
# }

# def testar_imagem():
#     # Carregar o modelo TFLite
#     interpreter = tf.lite.Interpreter(model_path="modelo_animais.tflite")
#     interpreter.allocate_tensors()

#     input_details = interpreter.get_input_details()
#     output_details = interpreter.get_output_details()

#     # Solicitar upload de imagem
#     print("Envie uma foto de um gato ou cachorro:")
#     uploaded = files.upload()

#     for fn in uploaded.keys():
#         # Carregar e processar a imagem
#         img = cv2.imread(fn)
#         img_resized = cv2.resize(img, (160, 160))

#         input_data = np.expand_dims(img_resized, axis=0).astype(np.float32)
#         input_data = (input_data / 127.5) - 1

#         # Fazer previsão
#         interpreter.set_tensor(input_details[0]['index'], input_data)
#         interpreter.invoke()
#         output_data = interpreter.get_tensor(output_details[0]['index'])

#         # Obter resultado
#         prediction = np.argmax(output_data)
#         animal_detectado = NOME_CLASSES[prediction]
#         confianca = 100 * np.max(output_data)

#         # Exibir resultado
#         display(Image(fn, width=200))
#         print(f"\n--- RESULTADO DA IA ---")
#         print(f"Identificado: {animal_detectado.upper()}")
#         print(f"Confiança: {confianca:.2f}%")
#         print(f"\nSOBRE O ANIMAL:")
#         print(INFO_ANIMAIS[animal_detectado])
#         print("-" * 30)

# # Executar
# testar_imagem()

================================================================================

**3. Treinamento do Modelo de Raças**

Este é diferente, você consegue colocar a imagem do cachorro ou gato para que a aplicação faça a identificação das raças. Nesse caso, utilizei o Oxford-IIIT Pet, que possui 37 raças diferentes de gatos e cachorros.

**OBS: A aplicação só vai identificar corretamente as raças as quais foi treinada.**

**3.1 Carregamento e Preparação**

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Carregar dataset de raças (Oxford-IIIT Pet)
print("Baixando dataset de raças...")
dataset_name = "oxford_iiit_pet"

(train_ds, validation_ds), metadata = tfds.load(
    name=dataset_name,
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True
)

nomes_das_racas = metadata.features['label'].names
num_classes = metadata.features['label'].num_classes

print(f"Total de raças: {num_classes}")
print(nomes_das_racas)

**3.2 Preparação dos Dados**

Padronizando novamente as imagens para não ter problema com os tamanhos diferentes.

In [9]:
IMG_SIZE = 160
BATCH_SIZE = 32

def format_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = (image / 127.5) - 1
    return image, label

train_batches = train_ds.map(format_image).shuffle(1000).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_ds.map(format_image).batch(BATCH_SIZE).prefetch(1)

**3.3 Criação e Treinamento**

Segue o mesmo padrão do primeiro exemplo de apenas identificar o pet.

In [ ]:
# Criar modelo
base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)  # 37 raças

model = Model(inputs=base_model.input, outputs=predictions)

# Compilar
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Treinar
print("Iniciando treinamento das raças...")
history = model.fit(
    train_batches,
    epochs=10,
    validation_data=validation_batches
)

# Converter para TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('modelo_racas.tflite', 'wb') as f:
    f.write(tflite_model)

print("Arquivo 'modelo_racas.tflite' criado!")

**4. Refinamento do Modelo (Fine Tuning)**

**4.1 Aumentar Inteligência com Fine Tuning**

Basicamente utilizando o Fine Tuning foi possivel continuar do treinamento anterior de apenas identificar se é gato ou cão e agora treinar mais um pouco para identificar as raças.

In [ ]:
# Descongelar o modelo base
base_model.trainable = True

# Congelar apenas as primeiras 100 camadas
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

print(f"Modelo base tem {len(base_model.layers)} camadas")

**4.2 Recompilar com Taxa de Aprendizado Baixa**

Por que os pesos já estão bons e eu só quero melhorar um pouco mais. Então, ele segue de onde parou.

In [ ]:
# Taxa de aprendizado MUITO baixa para fine tuning
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-5),
    metrics=['accuracy']
)

# Treinar mais um pouco
print("Iniciando ajuste fino (Fine Tuning)...")
history_fine = model.fit(
    train_batches,
    epochs=10,
    initial_epoch=history.epoch[-1],
    validation_data=validation_batches
)

**4.3 Salvar Versão Final**

In [ ]:
# Converter para TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('modelo_racas_pro.tflite', 'wb') as f:
    f.write(tflite_model)

print("Arquivo 'modelo_racas_pro.tflite' criado!")
print("Esta é a versão mais inteligente do modelo.")

**5. Teste do Modelo de Raças**

In [14]:
# import tensorflow as tf
# import numpy as np
# import cv2
# from google.colab import files
# from IPython.display import Image, display

# # Lista de 37 raças
# RACAS = [
#     'Abyssinian', 'American Bulldog', 'American Pit Bull Terrier', 'Basset Hound',
#     'Beagle', 'Bengal', 'Birman', 'Bombay', 'Boxer', 'British Shorthair',
#     'Chihuahua', 'Egyptian Mau', 'English Cocker Spaniel', 'English Setter',
#     'German Shorthaired', 'Great Pyrenees', 'Havanese', 'Japanese Chin',
#     'Keeshond', 'Leonberger', 'Maine Coon', 'Miniature Pinscher', 'Newfoundland',
#     'Persian', 'Pomeranian', 'Pug', 'Ragdoll', 'Russian Blue', 'Saint Bernard',
#     'Samoyed', 'Scottish Terrier', 'Shiba Inu', 'Siamese', 'Sphynx',
#     'Staffordshire Bull Terrier', 'Wheaten Terrier', 'Yorkshire Terrier'
# ]

# def testar_raca():
#     # Carregar modelo
#     interpreter = tf.lite.Interpreter(model_path="modelo_racas.tflite")
#     interpreter.allocate_tensors()
#     input_details = interpreter.get_input_details()
#     output_details = interpreter.get_output_details()

#     print("Envie uma foto de um cachorro ou gato:")
#     uploaded = files.upload()

#     for fn in uploaded.keys():
#         # Processar imagem
#         img = cv2.imread(fn)
#         img_resized = cv2.resize(img, (160, 160))
#         input_data = np.expand_dims(img_resized, axis=0).astype(np.float32)
#         input_data = (input_data / 127.5) - 1

#         # Previsão
#         interpreter.set_tensor(input_details[0]['index'], input_data)
#         interpreter.invoke()
#         output_data = interpreter.get_tensor(output_details[0]['index'])

#         # Resultado
#         prediction_idx = np.argmax(output_data)
#         raca_detectada = RACAS[prediction_idx]
#         confianca = 100 * np.max(output_data)

#         # Exibir
#         display(Image(fn, width=200))
#         print(f"\n--- RESULTADO ---")
#         print(f"Raça: {raca_detectada}")
#         print(f"Confiança: {confianca:.2f}%")

#         # Identificar espécie
#         gatos = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British Shorthair',
#                  'Egyptian Mau', 'Maine Coon', 'Persian', 'Ragdoll', 'Russian Blue',
#                  'Siamese', 'Sphynx']

#         if raca_detectada in gatos:
#             print("Espécie: Gato")
#         else:
#             print("Espécie: Cachorro")

# # Executar
# testar_raca()

##A partir daqui você vai conseguir testar a aplicação como um todo, ou seja, web.

**6. Aplicação Streamlit**

Foi utilizado essa biblioteca por que permite com que construa aplicações web interativas sem precisar escrever HMTL, CSS ou JS.

**6.1 Instalação e Setup:**

In [ ]:
# Instalar dependências
!pip install -q streamlit pyngrok

**6.2 Criar Arquivo app.py**

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import cv2
import numpy as np
from PIL import Image

# --- CORREÇÃO AQUI ---
# Seu modelo pede 160, então vamos dar 160 pra ele.
IMG_SIZE_APP = 160
# Confirme se o nome do arquivo abaixo é o que está na sua pasta
MODEL_PATH = 'modelo_racas_pro.tflite'

# (O resto do código continua igual...)
RACAS = [
    'Abyssinian', 'American Bulldog', 'American Pit Bull Terrier', 'Basset Hound',
    'Beagle', 'Bengal', 'Birman', 'Bombay', 'Boxer', 'British Shorthair',
    'Chihuahua', 'Egyptian Mau', 'English Cocker Spaniel', 'English Setter',
    'German Shorthaired', 'Great Pyrenees', 'Havanese', 'Japanese Chin',
    'Keeshond', 'Leonberger', 'Maine Coon', 'Miniature Pinscher', 'Newfoundland',
    'Persian', 'Pomeranian', 'Pug', 'Ragdoll', 'Russian Blue', 'Saint Bernard',
    'Samoyed', 'Scottish Terrier', 'Shiba Inu', 'Siamese', 'Sphynx',
    'Staffordshire Bull Terrier', 'Wheaten Terrier', 'Yorkshire Terrier'
]

def get_info(raca):
    infos = {
        'Persian': "🐱 PERSA\nCuidados: Escovação diária.\nTemperamento: Calmo.",
        'Siamese': "🐱 SIAMÊS\nCuidados: Atenção constante.\nTemperamento: Falante.",
        'Boxer': "🐶 BOXER\nCuidados: Muito exercício.\nTemperamento: Brincalhão.",
        'Beagle': "🐶 BEAGLE\nCuidados: Passeios longos.\nTemperamento: Curioso.",
    }
    return infos.get(raca, f"Raça: {raca}\nRecomendação: Mantenha vacinação em dia e muito amor.")

# --- INTERFACE VISUAL ---
st.set_page_config(page_title="Pet ID", page_icon="🐾")
st.title("🐾 Identificador de Pets")
st.write("Formatos: JPG, PNG, WEBP, BMP, TIFF, JFIF.")

uploaded_file = st.file_uploader(
    "Escolha uma imagem...",
    type=["jpg", "png", "jpeg", "webp", "bmp", "tiff", "tif", "jfif"]
)

if uploaded_file is not None:
    try:
        image = Image.open(uploaded_file)
        st.image(image, caption='Imagem carregada', use_column_width=True)

        with st.spinner('Processando...'):
            image = image.convert('RGB')
            img_array = np.array(image)

            # REDIMENSIONANDO PARA 160 (Para bater com o modelo)
            img_resized = cv2.resize(img_array, (IMG_SIZE_APP, IMG_SIZE_APP))

            input_data = np.expand_dims(img_resized, axis=0).astype(np.float32)
            input_data = (input_data / 127.5) - 1

            interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
            interpreter.allocate_tensors()
            input_details = interpreter.get_input_details()
            output_details = interpreter.get_output_details()

            interpreter.set_tensor(input_details[0]['index'], input_data)
            interpreter.invoke()
            output_data = interpreter.get_tensor(output_details[0]['index'])

            idx = np.argmax(output_data)
            raca_detectada = RACAS[idx]
            confianca = 100 * np.max(output_data)

        st.success(f"Resultado: **{raca_detectada.upper()}**")
        st.progress(int(confianca))
        st.caption(f"Confiança: {confianca:.1f}%")
        st.info(get_info(raca_detectada))

    except Exception as e:
        st.error(f"Erro técnico: {e}")

**6.3 Colocar a aplicação online:**

In [ ]:
from pyngrok import ngrok
import time

# 1. Limpar processos antigos
!pkill -f streamlit

# --- 2. CONFIGURAÇÃO DO TOKEN (Obrigatório) ---
# Cole o código que você copiou do site DENTRO das aspas abaixo:
meu_token = "37uDfn5pJh1ddJs4FVOBvbaeYET_2oSW4BoVibmfNqFSWbdXD"

# Aplica o token no sistema
if meu_token == "COLE_SEU_TOKEN_AQUI":
    print("ERRO: Você esqueceu de colar seu token na linha acima!")
else:
    !ngrok config add-authtoken {meu_token}

    # 3. Criar o Link
    try:
        # Tenta conectar na porta 8501
        url_publica = ngrok.connect(8501).public_url
        print("="*50)
        print(f"SUCESSO! SEU APP ESTÁ AQUI: {url_publica}")
        print("Copie esse link e abra no seu celular.")
        print("="*50)
    except Exception as e:
        print("Ainda deu erro. Verifique se copiou o token inteiro.")
        print(e)

    # 4. Rodar o App (Isso fica rodando pra sempre, não pare essa célula)
    !streamlit run app.py &>/dev/null